# Reverse Power Hosting Capacity for a New Generator

In this example, we calculate the hosting capacity for a new generator at bus "bus_1101" using the Reverse Power metric. This example uses Python to control OpenDSS.

1. OpenDSS: <https://www.epri.com/pages/sa/opendss>
2. py-dss-interface: <https://pypi.org/project/py-dss-interface/>

Importing third-party Python packages and modules from this repository.

In [2]:
# -*- coding: utf-8 -*-
# @Author  : Paulo Radatz
# @Email   : pradatz@epri.com

In [3]:
import py_dss_interface
import numpy as np

In [4]:
from FeederCondition import FeederCondition
from HCSteps import HCSteps
from HCSeetings import HCSettings

## Feeder Model

Hosting capacity analysis requires a feeder model. You can find the feeder used in this example at <https://github.com/rkerestes/IEEE1729TestSystems/tree/main/HCA/HCABase.dss>

In [5]:
dss_file = r"C:\PauloRadatz\GitHub\IEEE1729TestSystems\HCA\HCABase.dss"

In [6]:
dss = py_dss_interface.DSS()
dss.text(f"compile [{dss_file}]")
dss.text("edit EnergyMeter.m2 enabled=no")

''

## Bus Selection

The bus selection is indicated below.

In [7]:
bus = "bus_1101"

## Load and Connected Generation Models

In this example, generator and load models are considered constant power. You can change this in OpenDSS if desired.

In [8]:
FeederCondition.set_load_model(dss)
FeederCondition.set_generator_model(dss)

## Feeder Condition | Hosting Capacity Impact Factors
Reverse power hosting capacity should be based on conditions of maximum generation output coinciding with minimum gross load on the primary circuit. It can be measured in specific system equipment, such as protective elements, voltage regulation equipment, substation bus, and/or other equipment. The maximum generation includes the maximum output of the new generator plus any connected generators. On the other hand, the minimum load is defined as the lowest load consumption across the entire timeframe. If more information on the coincidence of load consumption, connected generators' generation, and the new generator's generation is available, it could be taken into account to set the maximum generation and minimum load accordingly.

In this case, the connected generators are assumed to generate at 100% capacity, and the minimum load is considered to be 20% of the total load.

Besides the generator and load level impact factors, in this example, we can also consider whether the capacitor is ON or OFF. We can assume the capacitor is OFF, as the minimum demand condition typically results in switched capacitors being OFF.

In [9]:
hc_settings = HCSettings(load_mult=0.2, capacitor=False, add_existing_ger=True, mult_existing_gen=1)

In [10]:
FeederCondition.set_capacitor_status(dss, capacitor=hc_settings.capacitor)
FeederCondition.set_load_level_condition(dss, load_mult=hc_settings.load_mult)
FeederCondition.consider_existing_gen(dss, hc_settings.add_existing_ger)

## Adding a New Generator to the Feeder Model

We add a generator model as constant power to the feeder model. In this example, the generator is 3-phase.

In [11]:
dss.circuit.set_active_bus(bus)
kv = dss.bus.kv_base * np.sqrt(3)
gen_bus = {"gen": dss.bus.name}
gen_kv = {"gen": kv}
HCSteps.add_gen(dss, gen_bus, gen_kv)

## Hosting Capacity Iterative Process

The hosting capacity iterative process gradually increases the new generator's size and checks for violations. The process ends if there's a reverse power violation at the equipment being considered—in this case, only the substation bus—or if the generator size reaches the maximum analyzed size of 10 MW. A reverse power violation occurs when the sum of each phase's active power is negative, meaning active power flows from the feeder to the substation.

Thus, the hosting capacity corresponds to the generator size from the previous iteration before the violation.

In [12]:
i = 0
while i * HCSteps.step_kw < HCSteps.max_kw:
    i = i + 1
    i_kw = i * HCSteps.step_kw
    gen_kw = {"gen": i_kw}

    # Set Penetration Level
    HCSteps.increase_gen(dss, gen_kw)

    # Perform Power flow
    HCSteps.solve_powerflow(dss)

    # Violation?
    if HCSteps.check_reverse_power_violation(dss):
        hosting_capacity_value_kw = (i - 1) * HCSteps.step_kw
        break

In [13]:
print(f"Hosting Capacity = {hosting_capacity_value_kw / 1000} MW")

Hosting Capacity = 0.28 MW


## Hosting Capacity Result

This example demonstrates how to calculate reverse power flow hosting capacity using OpenDSS controlled by Python. Note that the hosting capacity result is strongly related to the feeder conditions considered for the analysis. In this case, the conditions are a load level of 20%, all generators at 100% output, and capacitors off. This scenario is likely the worst condition, and the feeder might not even operate under it. Therefore, to better represent the worst condition the feeder can actually operate under, more operational information is needed. This includes control elements, load comsumption, and generator production coincidence, which can then be considered in the hosting capacity analysis. However, sometimes this information is not available or known.

More impact factors can be considered in the analysis, such as smart inverter functionalities.

The model itself can also significantly impact the hosting capacity results. For instance, whether the lines are modeled as positive and negative sequences, which load and generation models are considered, etc.